# 01. Import & Install Dependencies

In [9]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2                                # Computer Vison Library OpenCV
import numpy as np                        # numpy arrays structure our datasets (numpy instead of tensorflow)
import os                                 # work with operating systems file path
from matplotlib import pyplot as plt      # matplotlib just helps us visualize images a little bit easier
import mediapipe as mp                    # mediapipe (https://google.github.io/mediapipe/)
from math import *

# 02. Using Mediapipe (Google's open-source framework)

In [2]:
mp_face_mesh = mp.solutions.face_mesh              # FaceMesh Model has -- 478 -- Keypoints 
mp_drawing_styles = mp.solutions.drawing_styles    # Drawing styles
mp_drawing = mp.solutions.drawing_utils            # Drawing utilites

## Given the original image and obtaining the values through the model.

In [3]:
def mediapipe_detection(image,model):                 
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                     # Image is no longer writeable
    results = model.process(image)                    # Make prediction
    image.flags.writeable = True                      # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # COLOR CONVERSION RGB 2 BGR
    return image, results

## Storing the keypoints values in the mediapipe into a variable.

In [4]:
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

## Calculating blink rate (Euclidean distance).

In [5]:
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio

# Euclaidean distance
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = sqrt((x1 - x)**2 + (y1 - y)**2)
    if 1> distance:
        return 1
    else:
        return distance


# 03. Extract Position Landmarks (Keypoints Values) 

In [89]:
def extract_landmarks(img, results):
    frame = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    mesh_coords = mesh_coord_detection(img, results)
    
    # extract face 468 keyponits
    face = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks[0] else np.zeros(478*3)
    
    # extract right-eye 16 keypoints
    right_eye_coords = np.array([mesh_coords[p] for p in RIGHT_EYE]).flatten()  
    
    # extract left-eye 16 keypoints
    left_eye_coords = np.array([mesh_coords[p] for p in LEFT_EYE]).flatten()    
    
    # calculating eye ratio
    ratio = np.array(blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)).flatten()  
    
    # return all feature values
#     return np.concatenate([face,right_eye_coords,left_eye_coords,ratio])
    return np.concatenate([face,ratio])


def mesh_coord_detection(img, results):
    if img is None:
        img_height, img_width= 0,0
    else:
        img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark] if results.multi_face_landmarks[0] else np.zeros(478 *2)

    # returning the list of tuples for each landmarks 
    return mesh_coord

In [82]:
468*3

1404

# 04. Setup Folder for Collection

In [83]:
DATA_PATH = os.path.join('ML_Data')         # Path for exported data, numpy arrays 
# C:\Users\Sms Studio\Desktop\University_Research_Projects\LieDetection\Semester Research Program\ML_Data
actions = np.array(['true', 'lie'])       # Actions that we try to detect
                      
true_vdos = 156                        # for taking total true videos.

lie_vdos = 64                        # for taking total lie videos.

sequence_length = 30             # Videos are going to be 30 frames in length

In [84]:
# Create Folders for save keypoints values

for action in actions:
    if action == 'true': 
        for sequence in range(true_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass
    else:
        for sequence in range(lie_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass

#  05. Collect Keypoints Values for Training and Testing

In [85]:
for action in actions:   # Loop through actions
    if action == 'true':
        for sequence in range(true_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/true/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read() # return status(boolean) , image                               # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/true/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face , ratio

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path

                    # Keypoints Shape:(1435,) ---> (1434 + 1)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values

                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______") 
    else:
        for sequence in range(lie_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/lie/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read()                                # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/lie/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face keypoints

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path

                    # Keypoints Shape:(1405,) ---> (1404 + 1)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values

                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______")

Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing

Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing Video Path: Datasets/true/6/6.mp4
Processing

Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Video Path: Datasets/true/12/12.mp4
Processing Vi

Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Video Path: Datasets/true/18/18.mp4
Processing Vi

Processing Video Path: Datasets/true/23/23.mp4
Processing Video Path: Datasets/true/23/23.mp4
Processing Video Path: Datasets/true/23/23.mp4
Processing Video Path: Datasets/true/23/23.mp4
Processing Video Path: Datasets/true/23/23.mp4
Processing Video Path: Datasets/true/23/23.mp4
_____Finish_______
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/24.mp4
Processing Video Path: Datasets/true/24/2

...

Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/lie/36/36.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/lie/42/42.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/lie/48/48.mp4
Processing Video Path: Datasets/li

Processing Video Path: Datasets/lie/53/53.mp4
_____Finish_______
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Video Path: Datasets/lie/54/54.mp4
Processing Vide

_____Finish_______
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Video Path: Datasets/lie/60/60.mp4
Processing Vide

In [86]:
feature_values = extract_landmarks(image,results)

In [87]:
feature_values.shape

(1435,)

In [88]:
478*3

1434

# 6. Preprocess Data and Create Labels and Features

In [90]:
from sklearn.model_selection import train_test_split # Scikit-learn for our evaluation metrics
from tensorflow.keras.utils import to_categorical   # Converting Data (Encoded)

In [91]:
label_map = {label:num for num, label in enumerate(actions)}  # Actions Label Map  {'true': 0, 'lie': 1} 

In [92]:
sequences, labels = [], []                 # like : Features(X) Data , labels(Y) Data
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []                            # Save all of diff.frames paricular sequence
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))   # load np array
            window.append(res)
            print('Processing Video : '+'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4')
        sequences.append(window)
        labels.append(label_map[action])

Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video

Processing Video : Datasets/true/103/103.mp4
Processing Video : Datasets/true/103/103.mp4
Processing Video : Datasets/true/103/103.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing Video : Datasets/true/104/104.mp4
Processing

Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/11/11.mp4
Processing Video : Datasets/true/110/110.mp4
Processing Video : Datasets/true/110/110.mp4
Processing Video : Datasets/true/110/110.mp4
Processing Video : Datasets/true/110/110.mp4
Processing Video : Datasets/true/110/110.mp4
P

Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/115/115.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing Video : Datasets/true/116/116.mp4
Processing

Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/121/121.mp4
Processing Video : Datasets/true/122/122.mp4
Processing Video : Datasets/true/122/122.mp4
Processing Video : Datasets/true/122/122.mp4
Processing Video : Datasets/true/122/122.mp4
Processing

...

Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/36/36.mp4
Processing Video : Datasets/lie/37/37.mp4
Processing Video : Datasets/lie/37/37.mp4
Processing Video : Datasets/lie/37/37.mp4
Processing Video : Datasets/lie/37

Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/42/42.mp4
Processing Video : Datasets/lie/43

Processing Video : Datasets/lie/48/48.mp4
Processing Video : Datasets/lie/48/48.mp4
Processing Video : Datasets/lie/48/48.mp4
Processing Video : Datasets/lie/48/48.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49/49.mp4
Processing Video : Datasets/lie/49

Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/54/54.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55/55.mp4
Processing Video : Datasets/lie/55

Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61/61.mp4
Processing Video : Datasets/lie/61

In [93]:
X = np.array(sequences)  # sequences
y = to_categorical(labels).astype(int)  # labels

# Train & Testing Partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

In [94]:
np.array(sequences).shape

(220, 30, 1435)

In [281]:
# X.shape : (220, 30, 1435)  -----> ( Sequences(videos) , framers per vdo, keypoints shape [(478 * 3) + 1] )
# y.shape : (220, 2) -------->   ( Sequences(videos), labels[actions] )

# X_train.shape : (209, 30, 1435) 
# X_test.shape : (11, 30, 1435)

# y_train.shape : (209, 2) 
# y_test.shape : (11, 2) 

In [95]:
X.shape

(220, 30, 1435)

# 7. Build and Train LSTM Neural Network

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [97]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [98]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1435)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [70]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [71]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
7/7 [==============================] - 13s 1s/step - loss: 3.7282 - categorical_accuracy: 0.5694
Epoch 2/100
7/7 [==============================] - 1s 105ms/step - loss: 11.3625 - categorical_accuracy: 0.6459
Epoch 3/100
7/7 [==============================] - 1s 100ms/step - loss: 23.6545 - categorical_accuracy: 0.5502
Epoch 4/100
7/7 [==============================] - 1s 111ms/step - loss: 33.8755 - categorical_accuracy: 0.6986
Epoch 5/100
7/7 [==============================] - 1s 110ms/step - loss: 22.4061 - categorical_accuracy: 0.6794
Epoch 6/100
7/7 [==============================] - 1s 105ms/step - loss: 47.6016 - categorical_accuracy: 0.5263
Epoch 7/100
7/7 [==============================] - 1s 103ms/step - loss: 42.4280 - categorical_accuracy: 0.6651 0s - loss: 45.2115 - categorical_accuracy: 0.
Epoch 8/100
7/7 [==============================] - 1s 100ms/step - loss: 17.7903 - categorical_accuracy: 0.5885
Epoch 9/100
7/7 [==============================] - 1s 98ms/st

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            384000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total params: 538,530
Trainable params: 538,530
Non-trainable params: 0
________________________________________________

# 8. Make Predictions

In [99]:
res = model.predict(X_test)

In [100]:
X_test.shape

(11, 30, 1435)

In [101]:
np.sum(res[0])

1.0

In [102]:
actions[np.argmax(res[2])]

'lie'

In [103]:
y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0]])

In [30]:
actions[np.argmax(y_test[10])]

'true'

# 9. Save Weights

In [31]:
model.save('action.h5')

In [211]:
del model

In [32]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [34]:
yhat = model.predict(X_test)

In [35]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [36]:
ytrue

[1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0]

In [37]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 4],
        [0, 6]],

       [[6, 0],
        [4, 1]]], dtype=int64)

In [38]:
accuracy_score(ytrue, yhat)

0.6363636363636364

# 11. Test in Real Time

In [172]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(
            output_frame, actions[num].capitalize() + ' ' + str(int(prob*100)) +' %', 
            (3, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [176]:
# New detection variables
sequence = []
threshold = 0.5

cap = cv2.VideoCapture('testing.mp4')

with mp.solutions.face_mesh.FaceMesh(      # set mediapipe model
    max_num_faces =1,
    refine_landmarks = True,   
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face_mesh)
        print(results)

        # Prediction logic
        keypoints = extract_landmarks(image, results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        
        if len(sequence) == 30:
            print(np.array(sequence).shape)
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
    
        # Show to screen
        cv2.imshow('Lie Detection', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(30, 1435)
lie
<class 'mediapipe.python.solution_base.SolutionOu

In [175]:
cap.release()
cv2.destroyAllWindows()